In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install imageio==2.4.1

In [ ]:
#installing paddleocr
!pip install "paddleocr>=2.0.1"

In [ ]:
!pip install paddlepaddle

In [ ]:
from paddleocr import PaddleOCR,draw_ocr
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory

ModuleNotFoundError: ignored

In [ ]:
#path to video
path='/content/drive/MyDrive/video.mp4'

In [ ]:
#mention the time interval to process
import cv2
cap = cv2.VideoCapture(path)
fps = cap.get(cv2.CAP_PROP_FPS)
start=int(60*3*fps)
end  =int(start+60*10*fps)

In [ ]:
start,end

In [ ]:
!rm -rf frames/*
!mkdir frames

In [ ]:
ls


In [ ]:
#reading video and saving frames to folder for every 2 sec
import cv2
 
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
 
cnt=1 

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    if cnt>=start and cnt<=end:

      if cnt%60==0:

        print("saved frame no. "+str(cnt))

        #Display the resulting frame
        cv2.imwrite('frames/'+str(cnt)+".png",frame)
  
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

    if cnt>end:
      break
 
  # Break the loop
  else: 
    break
  
  cnt=cnt+1

# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

In [ ]:
import os,re
names= os.listdir("frames")
names.sort(key=lambda f: int(re.sub('\D', '', f)))

In [ ]:
names[:5]

['5400.png', '5460.png', '5520.png', '5580.png', '5640.png']

In [ ]:
import numpy as np
from tqdm import tqdm
name_list=[]
text_list=[]
conf_list=[]

for i in tqdm(np.arange(0,len(names))):
  img_path = 'frames/'+names[i]
  result = ocr.ocr(img_path, cls=True)
  
  #print(result)
  #print("---------")

  for idx in range(len(result)):
    res = result[idx]
    for line in res:
      text,conf= line[1]
      name_list.append(names[i][:-4])
      text_list.append(text)
      conf_list.append(conf)

In [ ]:
import pandas as pd
df=pd.DataFrame({'name':name_list,"text":text_list,"conf":conf_list})

In [ ]:
df.head()

NameError: ignored

In [ ]:
text_df= df.groupby('name').apply(lambda x:" ".join(x['text'])).reset_index()

NameError: ignored

In [ ]:
text_df['scorecard']=text_df[0].apply(lambda x:re.findall(r"AUS \d{1,3}/\d{1,3}",x))

In [ ]:
ball_df= text_df[text_df['scorecard'].apply(lambda x:True if len(x)>0 else False)]

In [ ]:
ball_df

,name,0,scorecard
0,10020,TPOWER AUS 0/0 TORACE EHD OVER0.1,[AUS 0/0]
1,10080,AUS 0/0 OVER0.1,[AUS 0/0]
2,10140,AUS 0/0 OSHD OVER0.1,[AUS 0/0]
4,10260,AUS 0/0 AEADE HD OVER0.1,[AUS 0/0]
5,10320,AUS 0/0 OVERO. PEST,[AUS 0/0]
...,...,...,...
275,9540,AUS 0/0 HD OVERO PRICK,[AUS 0/0]
278,9720,POWERADE AUS 0/0 OVER0 yEmiras FlyEmirates,[AUS 0/0]
279,9780,POWERADE AUS 0/0 HD OVER0 irates FlyEmirates,[AUS 0/0]
280,9840,AUS 0/0 HD OVERO.,[AUS 0/0]


In [ ]:
ball_df['score']=ball_df['scorecard'].apply(lambda x:re.findall(r"\d{1,3}/\d{1,3}",x[0])[0])

In [ ]:
ball_df

,name,0,scorecard,score
0,10020,TPOWER AUS 0/0 TORACE EHD OVER0.1,[AUS 0/0],0/0
1,10080,AUS 0/0 OVER0.1,[AUS 0/0],0/0
2,10140,AUS 0/0 OSHD OVER0.1,[AUS 0/0],0/0
4,10260,AUS 0/0 AEADE HD OVER0.1,[AUS 0/0],0/0
5,10320,AUS 0/0 OVERO. PEST,[AUS 0/0],0/0
...,...,...,...,...
275,9540,AUS 0/0 HD OVERO PRICK,[AUS 0/0],0/0
278,9720,POWERADE AUS 0/0 OVER0 yEmiras FlyEmirates,[AUS 0/0],0/0
279,9780,POWERADE AUS 0/0 HD OVER0 irates FlyEmirates,[AUS 0/0],0/0
280,9840,AUS 0/0 HD OVERO.,[AUS 0/0],0/0


In [ ]:
ball_df['over']=ball_df[0].apply(lambda x:re.findall(r"OVER\s?[0-9\.]+",x))

In [ ]:
ball_df= ball_df[ball_df['over'].apply(lambda x:True if len(x)!=0 else False)]

In [ ]:
ball_df['over_num']=ball_df['over'].apply(lambda x:x[0])

In [ ]:
ball_df['over_num']=ball_df['over_num'].apply(lambda x:re.findall("[0-9\.]+",x)[0])

In [ ]:
ball_df['name']=ball_df['name'].astype(int)

In [ ]:
ball_df

,name,0,scorecard,score,over,over_num
0,10020,TPOWER AUS 0/0 TORACE EHD OVER0.1,[AUS 0/0],0/0,[OVER0.1],0.1
1,10080,AUS 0/0 OVER0.1,[AUS 0/0],0/0,[OVER0.1],0.1
2,10140,AUS 0/0 OSHD OVER0.1,[AUS 0/0],0/0,[OVER0.1],0.1
4,10260,AUS 0/0 AEADE HD OVER0.1,[AUS 0/0],0/0,[OVER0.1],0.1
6,10380,AUS 0/0 OVER0. AUS WON THE TOSS &ELECTED TOBAT...,[AUS 0/0],0/0,[OVER0.],0.
...,...,...,...,...,...,...
218,23220,AUS 0/19 OVER2.1,[AUS 0/19],0/19,[OVER2.1],2.1
219,23280,AUS 0/19 OVER2.1,[AUS 0/19],0/19,[OVER2.1],2.1
278,9720,POWERADE AUS 0/0 OVER0 yEmiras FlyEmirates,[AUS 0/0],0/0,[OVER0],0
279,9780,POWERADE AUS 0/0 HD OVER0 irates FlyEmirates,[AUS 0/0],0/0,[OVER0],0


In [ ]:
ball_df= ball_df.sort_values(['name'])

In [ ]:
ball_df['wicket']=ball_df['score'].apply(lambda x:x.split("/")[0])

In [ ]:
ball_df['score1']=ball_df['score'].apply(lambda x:x.split("/")[1])

In [ ]:
final_df = ball_df[['name','over_num','score1','wicket']]

In [ ]:
final_df[['name','score1','wicket']]=final_df[['name','score1','wicket']].astype(int)
final_df['over_num']=final_df['over_num'].astype(float)

In [ ]:
final_df= final_df.rename(columns={'score1':'score'})

In [ ]:
#need to take the first instance in over_num
temp=final_df[final_df['over_num']==0.5]
temp

,name,over_num,score,wicket
76,14580,0.5,6,0
77,14640,0.5,7,0
78,14700,0.5,7,0
79,14760,0.5,7,0
80,14820,0.5,7,0
82,14940,0.5,7,0
83,15000,0.5,7,0
89,15360,0.5,7,0
91,15480,0.5,7,0


In [ ]:
indices= final_df.groupby('over_num').apply(lambda x:x['score'].idxmax()).reset_index()[0].values
final_df= final_df.loc[indices]

In [ ]:
final_df = final_df.sort_values(['over_num','score'])

In [ ]:
score= final_df['score'].values
diff= score[1:]-score[:-1]
indices= np.where(diff>=4)[0]

In [ ]:
boundaries_df= final_df.iloc[indices+1]

In [ ]:
wicket= final_df['wicket'].values
diff= wicket[1:]-wicket[:-1]
indices= np.where(diff==1)[0]

In [ ]:
wickets_df= final_df.iloc[indices+1]

In [ ]:
highlights_df= pd.concat([boundaries_df,wickets_df])

In [ ]:
highlights_df= highlights_df.sort_values(['name'])

In [ ]:
highlights_df

,name,over_num,score,wicket
77,14640,0.5,7,0
102,16140,0.1,8,0
129,17760,1.2,12,0
172,20340,1.0,13,0
183,21000,1.5,17,0


In [ ]:
highlights_df['name']= highlights_df['name']

In [ ]:
highlights_df['time']=highlights_df['name']/fps

In [ ]:
highlights_df['time']= highlights_df['time'].astype(int)

In [ ]:
highlights_df

,name,over_num,score,wicket,time
77,14640,0.5,7,0,488
102,16140,0.1,8,0,538
129,17760,1.2,12,0,592
172,20340,1.0,13,0,678
183,21000,1.5,17,0,700


In [ ]:
time_list= highlights_df['time'].values

In [ ]:
!rmdir highlights
!mkdir highlights

In [ ]:
%%time
cnt=1
for end in time_list:
  start=end-10
  name=str(cnt)+".mp4"
  !ffmpeg -y -i $path -ss $start -t 10 $name
  cnt=cnt+1

In [ ]:
!rm merge.txt
!touch merge.txt

rm: cannot remove 'merge.txt': No such file or directory


In [ ]:
for i in range(len(highlights_df)):
  name='file'+' '+str(i+1)+".mp4"
  !echo $name >> merge.txt

In [ ]:
!ffmpeg -f concat -i merge.txt -c copy highlights.mp4

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [ ]:
from google.colab import files
files.download("highlights.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>